In [1]:
# taking code from
# https://github.com/jowagner/CA4023-NLP/blob/main/notebooks/sentiment-bert.ipynb

## 1.1 BERT Configuration

In [2]:
model_size          = 'base'  # choose between 'tiny', 'base' and 'large'
max_sequence_length = 256
batch_size          = 8       # 10 should work on a 12 GB card if not also used for graphics / GUI
virtual_batch_size  = 64

# TODO: what virtual batch size should we use
# https://arxiv.org/abs/1904.00962 are cited to say a large batch size (32k) is better
# but they themselves warn that "naively" doing so can degrade performance

max_epochs = 8
limit_train_batches = 1.0     # fraction of training data to use

hparams = {
    "encoder_learning_rate": 1e-05,  # Encoder specific learning rate
    "learning_rate":         3e-05,  # Classification head learning rate
    "nr_frozen_epochs":      3,      # Number of epochs we want to keep the encoder model frozen
    "loader_workers":        4,      # How many subprocesses to use for data loading.
                                     # (0 means that the data will be loaded in the main process)
    "batch_size":            batch_size,
    "gpus":                  1,
}  

training_task = 'Full'       # modify training and dev data as described below

assert training_task in [
    'Full',    # sequence B is the review sentence as is
    'SE',      # sequence B is only the sentiment expression, all other words are masked
    'Other',   # sequence B is all but the SE (the SE words are masked)
]
print('Ready')

Ready


In [3]:
# compensate for small batch size with batch accumulation if needed
accumulate_grad_batches = 1
while batch_size * accumulate_grad_batches < virtual_batch_size:
    # accumulated batch size too small
    # --> accumulate more batches
    accumulate_grad_batches += 1

print('Batch size:', batch_size)
if accumulate_grad_batches > 1:
    print('Accumulating gradients of %d batches' %accumulate_grad_batches)
    
size2name = {
    'tiny':  'distilbert-base-uncased',
    'base':  'bert-base-uncased',
    'large': 'bert-large-uncased',
}

model_name = size2name[model_size]

if training_task == 'Full':
    training_mask = None
elif training_task == 'SE':
    training_mask = 'O'
elif training_task == 'Other':
    training_mask = 'I'
else:
    raise ValueError('unknown training task %s' %training_task)
    
from transformers import AutoTokenizer
from tokenizers.pre_tokenizers import Whitespace

tokeniser = AutoTokenizer.from_pretrained(model_name)

Batch size: 8
Accumulating gradients of 8 batches


## 1.2 Dataset Configuration

In [4]:
domains = ['laptop', 'restaurant']
#domains = ['restaurant']

train_dev_split = (95, 5)

data_prefix = 'data/'

filenames = {
    'laptop':     (('ABSA16_Laptops_Train_SB1_v2.xml',
                    'train.laptop.aio'
                   ),
                   ('EN_LAPT_SB1_TEST_.xml.gold',
                    'test.laptop.aio'
                   )
                  ),
                   
    'restaurant': (('ABSA16_Restaurants_Train_SB1_v2.xml',
                    'train.restaurant.aio'
                   ),
                   ('EN_REST_SB1_TEST.xml.gold',
                    'test.restaurant.aio'
                   )
                  ),
}

for domain in domains:
    for part in (0,1):
        for filename in filenames[domain][part]:
            filename = data_prefix + filename
            print('Using', filename)

Using data/ABSA16_Laptops_Train_SB1_v2.xml
Using data/train.laptop.aio
Using data/EN_LAPT_SB1_TEST_.xml.gold
Using data/test.laptop.aio
Using data/ABSA16_Restaurants_Train_SB1_v2.xml
Using data/train.restaurant.aio
Using data/EN_REST_SB1_TEST.xml.gold
Using data/test.restaurant.aio


## 1.3 Question Templates

In [5]:
put_question_first = True  # whether to put question into seq A or B

#use_questions_with_labeltype = ['polarity', 'yesno']
use_questions_with_labeltype = ['polarity']
#use_questions_with_labeltype = ['yesno']

use_question_with_description = 'Sun et al. QA-M'    # set to None to de-activate this filter
#use_question_with_description = None

templates = [
    
    # Hoang et al. (2019)
    {   'question': '%(entity_type)s, %(attribute_label)s',
        'label':    '%(polarity)s',
        'description': 'Hoang et al.',
    },
    
    # Sun et al. (2019) format 1
    {   'question': '%(entity_type)s - %(attribute_label)s',
        'label':    '%(polarity)s',
        'description': 'Sun et al. NLI-M',
    },
    
    # Sun et al. (2019) format 2 "NLI-M"
    {    'question': 'What do you think of the %(attribute_label)s of %(entity_type)s?',
         'label':    '%(polarity)s',
         'description': 'Sun et al. QA-M',
    },
    
    # Sun et al. (2019) format 3 "QA-B"
    {    'question': 'The polarity of the aspect %(attribute_label)s of %(entity_type)s is %(candidate_polarity)s.',
         'label':    '%(yesno)s',
         'description': 'Sun et al. QA-B',
    },
    
    # Sun et al. (2019) format 4
    {    'question': '%(entity_type)s - %(attribute_label)s - %(candidate_polarity)s',
         'label':    '%(yesno)s',
         'description': 'Sun et al. NLI-B',
    },
]

In [6]:
templates += [
    
    # Variant 1
    {    'question': 'In terms of %(attribute_label)s, what do you think of %(entity_type)s?',
         'label':    '%(polarity)s',
         'description': 'Variant 1, QA-M',
    },
    
    # Variant 2
    {    'question': 'What polarity has the sentiment towards the %(attribute_label)s of %(entity_type)s in the following rewview?',
         'label':    '%(polarity)s',
         'description': 'Variant 2, QA-M',
    },
    
    # Variant 3
    {    'question': 'Do you agree that the sentiment towards the aspect %(attribute_label)s of %(entity_type)s in the following review is %(candidate_polarity)s?',
         'label':    '%(yesno)s',
         'description': 'Variant 3, QA-B',
    },
    
    # Variant 4
    {    'question': 'Is there %(candidate_polarity)s sentiment towards the %(attribute_label)s aspect of %(entity_type)s?',
         'label':    '%(yesno)s',
         'description': 'Variant 4, QA-B',
    },
    
]

# TODO: add variants with entity type and attribute label not in ALLCAPS and
#       with _ replaced with whitespace (requires additional code)

In [7]:
# remove templates with wrong question type

filtered_templates = []
for template in templates:
    use_template = False
    for labeltype in use_questions_with_labeltype:
        if labeltype in template['label']:
            use_template = True
            break
    if use_question_with_description:
        if use_question_with_description != template['description']:
            use_template = False
    if use_template:
        filtered_templates.append(template)
templates = filtered_templates
print(len(templates), 'template(s) selected')


# If there are multiple domains it probably helps to include the domain in the question
# TODO: empirically test this idea

if len(domains) > 1:    
    question_prefix = '%(domain)s: '
    for template in templates:
        template['question'] = question_prefix + template['question']
        
assert len(templates) > 0

1 template(s) selected


## 2.1 Get Data Instances from XML and AIO Files

In [8]:
def get_annotation(aio_filename):
    f = open(aio_filename, 'r')
    tokens = []
    sea = []
    line_no = 0
    while True:
        line = f.readline()
        line_no += 1
        if line.isspace() or not line:
            if tokens:
                yield (tokens, sea)
                tokens = []
                sea = []
            if not line:
                break
        else:
            fields = line.split()
            if len(fields) != 2:
                raise ValueError('Unexpected AIO line %d: %r' %(line_no, line))
            tokens.append(fields[0])
            sea.append(fields[1])
    f.close()
    
def get_alignment(text, annotation):
    tokens, sea = annotation
    text1 = text.replace(' ', '')
    text1 = text1.replace('\xa0', '')   # also remove non-breakable space
    text2 = ''.join(tokens)
    text2 = text2.replace('-EMTCN01-', ':)')  # restore emoticon in SEA
    if text1 != text2:
        print('Mismatch %r - %r' %(text, tokens))
    #else:
    #    print('Match %r - %r' %(text, tokens))
    return tokens, sea

def absa_sort_key(sentence_xml_element):
    sent_id = sentence_xml_element.get('id')
    fields = sent_id.split(':')
    assert len(fields) == 2
    return (fields[0], int(fields[1]))

In [9]:
# mostly implemented from scratch, some inspiration from
# https://opengogs.adaptcentre.ie/rszk/sea/src/master/lib/semeval_absa.py

from xml.etree import ElementTree

def get_dataset(
    xml_filename, aio_filename, domain,
    observed_entity_types, observed_attribute_labels,
    observed_polarities,   observed_targets
):
    xmltree = ElementTree.parse(xml_filename)
    xmlroot = xmltree.getroot()
    annotation = get_annotation(aio_filename)
    dataset = []
    for sentence in sorted(
        xmlroot.iter('sentence'), key = absa_sort_key
    ):
        sent_id = sentence.get('id')
        #print('sent_id', sent_id)
        # get content inside the first <text>...</text> sub-element
        text = sentence.findtext('text').strip()
        for opinion in sentence.iter('Opinion'):
            tokens, sea = get_alignment(text, annotation.__next__())
            opin_cat = opinion.get('category')
            #print('opin_cat', opin_cat)
            entity_type, attribute_label = opin_cat.split('#')
            polarity = opinion.get('polarity')
            target = opinion.get('target')
            try:
                span = (int(opinion.get('from')), int(opinion.get('to')))
            except TypeError:
                # at least one of 'from' or 'to' is missing
                span = (0, 0)
            if target == 'NULL':
                target = None
            # add to dataset
            dataset.append((
                domain,
                sent_id, text, tokens, sea,
                entity_type, attribute_label,
                target, span,
                polarity
            ))
            # update vocabularies
            observed_entity_types.add(entity_type)
            observed_attribute_labels.add(attribute_label)
            observed_polarities.add(polarity)
            if target:
                observed_targets.add(target)
        #print()
    return dataset

In [10]:
# get training data

tr_observed_entity_types = set()
tr_observed_attribute_labels = set()
tr_observed_polarities = set()
tr_observed_targets = set()

tr_dataset = []
for domain in domains:
    xml_filename = data_prefix + filenames[domain][0][0]
    aio_filename = data_prefix + filenames[domain][0][1]
    tr_dataset += get_dataset(
        xml_filename, aio_filename, domain,
        tr_observed_entity_types, tr_observed_attribute_labels,
        tr_observed_polarities,   tr_observed_targets
    )

print('dataset size:', len(tr_dataset))
print('\nobserved entity types:',     sorted(tr_observed_entity_types))
print('\nobserved attribute labels:', sorted(tr_observed_attribute_labels))
print('\nobserved polarities:',       sorted(tr_observed_polarities))
print('\nnumber of unique targets:',  len(tr_observed_targets))

dataset size: 5416

observed entity types: ['AMBIENCE', 'BATTERY', 'COMPANY', 'CPU', 'DISPLAY', 'DRINKS', 'FANS_COOLING', 'FOOD', 'GRAPHICS', 'HARDWARE', 'HARD_DISC', 'KEYBOARD', 'LAPTOP', 'LOCATION', 'MEMORY', 'MOTHERBOARD', 'MOUSE', 'MULTIMEDIA_DEVICES', 'OPTICAL_DRIVES', 'OS', 'PORTS', 'POWER_SUPPLY', 'RESTAURANT', 'SERVICE', 'SHIPPING', 'SOFTWARE', 'SUPPORT', 'WARRANTY']

observed attribute labels: ['CONNECTIVITY', 'DESIGN_FEATURES', 'GENERAL', 'MISCELLANEOUS', 'OPERATION_PERFORMANCE', 'PORTABILITY', 'PRICE', 'PRICES', 'QUALITY', 'STYLE_OPTIONS', 'USABILITY']

observed polarities: ['negative', 'neutral', 'positive']

number of unique targets: 721


In [11]:
# get test data

te_observed_entity_types = set()
te_observed_attribute_labels = set()
te_observed_polarities = set()
te_observed_targets = set()

te_dataset = []
for domain in domains:
    xml_filename = data_prefix + filenames[domain][1][0]
    aio_filename = data_prefix + filenames[domain][1][1]
    te_dataset += get_dataset(
        xml_filename, aio_filename, domain,
        te_observed_entity_types, te_observed_attribute_labels,
        te_observed_polarities,   te_observed_targets
    )

print('dataset size:', len(te_dataset))
print('\nnew observed entity types:',     sorted(te_observed_entity_types - tr_observed_entity_types))
print('\nnew observed attribute labels:', sorted(te_observed_attribute_labels - tr_observed_attribute_labels))
print('\nnew observed polarities:',       sorted(te_observed_polarities - tr_observed_polarities))
print('\nnumber of unique targets:',  len(te_observed_targets))

Mismatch '3. Cannot manually take battery out, unless you work for apple or have the tools' - ['3', '.', 'can', 'not', 'manually', 'take', 'battery', 'out', ',', 'unless', 'you', 'work', 'for', 'apple', 'or', 'have', 'the', 'tools']
Mismatch '3. Cannot manually take battery out, unless you work for apple or have the tools' - ['3', '.', 'can', 'not', 'manually', 'take', 'battery', 'out', ',', 'unless', 'you', 'work', 'for', 'apple', 'or', 'have', 'the', 'tools']
Mismatch 'Cannot even get to bios.' - ['can', 'not', 'even', 'get', 'to', 'bios', '.']
dataset size: 1660

new observed entity types: []

new observed attribute labels: []

new observed polarities: []

number of unique targets: 312


## 2.2 Training-Dev Split
The SemEval ABSA dataset comes without a dev set. We need a dev set to decide how long to train, to select other parameters and to select a good run.

In [12]:
import random

# how many instances are there for each label?

group2indices = {}
for index, item in enumerate(tr_dataset):
    domain   = item[0]
    polarity = item[-1]
    group = (domain, polarity)
    if not group in group2indices:
        group2indices[group] = []
    group2indices[group].append(index)

# create stratified sample
    
rel_train_size, rel_dev_size = train_dev_split  # configured in section 1.2
rel_total = rel_train_size + rel_dev_size

tr_indices = []
dev_indices = []

for group in group2indices:
    indices = group2indices[group]
    n = len(indices)
    select = (n * rel_train_size) // rel_total
    remaining = n - select
    print('%r: split %d (%.1f%%) to %d (%.1f%%)' %(
        group, select, 100.0*select/float(n),
        remaining, 100.0*remaining/float(n),
    ))
    random.shuffle(indices)
    tr_indices += indices[:select]
    dev_indices += indices[select:]

tr_indices.sort()
dev_indices.sort()

def get_subset(dataset, indices):  # TODO: this probably can be replaced with [] slicing
    retval = []
    for index in indices:
        retval.append(dataset[index])
    return retval
    
dev_dataset = get_subset(tr_dataset, dev_indices)
tr_dataset  = get_subset(tr_dataset, tr_indices)

print()
print('Training data size:', len(tr_dataset))
print('Development data size:', len(dev_dataset))

('laptop', 'positive'): split 1555 (95.0%) to 82 (5.0%)
('laptop', 'neutral'): split 178 (94.7%) to 10 (5.3%)
('laptop', 'negative'): split 1029 (94.9%) to 55 (5.1%)
('restaurant', 'negative'): split 711 (94.9%) to 38 (5.1%)
('restaurant', 'positive'): split 1574 (95.0%) to 83 (5.0%)
('restaurant', 'neutral'): split 95 (94.1%) to 6 (5.9%)

Training data size: 5142
Development data size: 274


## 2.3 PyTorch DataLoader

To use the PyTorch Lighting framwork, we need to distinguish 3 types of objects handling our data:

### Dataset

PyTorch Dataset objects provide access to a data set and behave like a list of dictionaries, one dictionary for each data instance (training or test item). The framework does not prescribe what the dictionaries look like, i.e. you can choose the keys. The length of the list determines the number of training instances in each epoch, unless the DataLoader (below) is extended to filter or augment the data. The standard way to augment data is to keep the number of instances identical to the number of raw instances and to apply a different or random transformation in each call of `__getitem__()`.

### DataLoader

PyTorch DataLoader objects shuffle data provided by a Dataset object and create batches of data.

### LightningDataModule

LightningDataModule objects create 3 DataLoader objects, one each for training, validation and test data.

In [13]:
# basic usage of pytorch and lightning from
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# and
# https://github.com/ricardorei/lightning-text-classification/blob/master/classifier.py

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler

class ABSA_Dataset_part_1(Dataset):
    
    def __init__(
        self,
        raw_data,
        put_question_first = True,
        template_index = -1,    # -1 = pick random template
        mask = None,            # 'I' = mask SE tokens, i.e. use other tokens only,
                                # 'O' = mask other tokens, i.e. use SE tokens only
        info = None,            # additional info to keep with each instance
    ):
        self.raw_data            = raw_data
        self.put_question_first  = put_question_first
        self.template_index      = template_index
        self.mask                = mask
        self.info                = info
        
    def __len__(self):
        return len(self.raw_data)
    
    def __getitem__(self, idx):
        ''' get one instance of the dataset as a custom dictionary
        '''
        if torch.is_tensor(idx):
            idx = idx.tolist()
            assert isinstance(idx, int)
        domain, sent_id, text, \
            tokens, sea, \
            entity_type, attribute_label, \
            target, span, \
            polarity = self.raw_data[idx]
        question, label = self.pick_question(
            entity_type, attribute_label, domain, polarity,
        )
        tokens = self.apply_mask(tokens, sea)
        # TODO: support adding context (previous sentences) to text
        retval = {}
        if self.put_question_first:
            retval['seq_A'] = question
            retval['seq_B'] = tokens
        else:
            retval['seq_A'] = tokens
            retval['seq_B'] = question
        retval['label'] = label
        retval['domain'] = domain
        retval['mask']  = self.mask
        retval['info']  = self.info
        return retval

In [14]:
import random

class ABSA_Dataset(ABSA_Dataset_part_1):
                   
    def pick_question(self, entity_type, attribute_label, domain, polarity):
        global templates
        global observed_polarities
        if self.template_index < 0:
            template = random.choice(templates)
        else:
            template = templates[self.template_index]
        candidate_polarity = random.choice(list(tr_observed_polarities))
        if candidate_polarity == polarity:
            yesno = 'yes'
        else:
            yesno = 'no'
        question = template['question'] %locals()
        label    = template['label']    %locals()
        # tokenise question
        # (no need to split off punctuation as BERT always
        #  splits off non-alphanumeric characters and we
        #  do not need to map back to tokens for the question)
        question = question.split()
        return (question, label)
    
    def apply_mask(self, tokens, sea):
        if not self.mask:
            return tokens
        retval = []
        for index, token in enumerate(tokens):
            if sea[index] == self.mask:
                retval.append('[MASK]')
            else:
                retval.append(token)
        return retval
    
# to test this class, go to Appendix C

In [15]:
# wrap training and dev data

tr_dataset_object = ABSA_Dataset(
    tr_dataset,
    put_question_first = put_question_first,
    template_index = -1,  # pick question at random
    mask = training_mask,
)

print('Training size:', len(tr_dataset_object))

# create a separate dev set for each question template

dev_dataset_objects = []
for template_index in range(len(templates)):
    dev_dataset_objects.append(ABSA_Dataset(
        dev_dataset,
        put_question_first = put_question_first,
        template_index = template_index,
        mask = training_mask,    # e.g. if we train on SE data we want to
                                 # pick the model according to the SE score
    ))

# also provide a dev set that is the union of the above dev sets,
# e.g. to be used for model selection

dev_dataset_combined = torch.utils.data.ConcatDataset(dev_dataset_objects)

print('Devset size (using all templates):', len(dev_dataset_combined))

Training size: 5142
Devset size (using all templates): 274


## 2.4 Test Data

In [16]:
# create a separate test set for each question template

te_dataset_objects = []
for mask in (None, 'O', 'I'):
    for template_index in range(len(templates)):
        te_dataset_objects.append(ABSA_Dataset(
            te_dataset,
            put_question_first = put_question_first,
            template_index = template_index,
            mask = mask,
        ))

# also provide a test set that is the union of the above test sets

te_dataset_combined = torch.utils.data.ConcatDataset(te_dataset_objects)

print('Test set size (using all templates):', len(te_dataset_combined))

Test set size (using all templates): 4980


## 2.5 Lightning Wrapper for Training, Development and Test Data

In [17]:
# https://github.com/ricardorei/lightning-text-classification/blob/master/classifier.py
    
import pytorch_lightning as pl
from torchnlp.encoders import LabelEncoder

class ABSA_DataModule(pl.LightningDataModule):
    
    def __init__(self, classifier, data_split = None, **kwargs):
        global use_questions_with_labeltype
        super().__init__()
        self.hparams.update(classifier.hparams)
        self.classifier = classifier
        if data_split is None:      # this happens when loading a checkpoint
            data_split = (None, None, None)
        self.data_split = data_split
        self.kwargs = kwargs
        labelset = set()
        if 'polarity' in use_questions_with_labeltype:
            for label in tr_observed_polarities:
                labelset.add(label)
        if 'yesno' in use_questions_with_labeltype:
            labelset.add('yes')
            labelset.add('no')
        print('Labelset:', labelset)
        self.label_encoder = LabelEncoder(
            sorted(list(labelset)),
            reserved_labels = [],
        )

    def train_dataloader(self) -> DataLoader:
        ''' create a data loader for the training data '''
        dataset = self.data_split[0]
        return DataLoader(
            dataset     = dataset,
            sampler     = RandomSampler(dataset),
            batch_size  = self.hparams.batch_size,
            collate_fn  = self.classifier.prepare_sample,
            num_workers = self.hparams.loader_workers,
        )
    
    def val_dataloader(self) -> DataLoader:
        ''' create a data loader for the validation data '''
        return DataLoader(
            dataset     = self.data_split[1],
            batch_size  = self.hparams.batch_size,
            collate_fn  = self.classifier.prepare_sample,
            num_workers = self.hparams.loader_workers,
        )
    
    def test_dataloader(self) -> DataLoader:
        ''' create a data loader for the test data '''
        return DataLoader(
            dataset     = self.data_split[2],
            batch_size  = self.hparams.batch_size,
            collate_fn  = self.classifier.prepare_sample,
            num_workers = self.hparams.loader_workers,
        )

## 3.1 Classifier

In [18]:
from transformers import AutoModel
import torch.nn as nn

class Classifier_part_1(pl.LightningModule):
    
    #def __init__(self, hparams = None, **kwargs) -> None:
    def __init__(self, hparams = None, **kwargs) -> None:
        super().__init__()
        if type(hparams) is dict:
            #print('Converting', type(hparams))
            hparams = pl.utilities.AttributeDict(hparams)
        #print('New classifier with', hparams)
        # https://discuss.pytorch.org/t/pytorch-lightning-module-cant-set-attribute-error/121125
        if hparams:
            self.hparams.update(hparams)
        self.add_missing_hparams()
        hparams = self.hparams
        self.batch_size = hparams.batch_size
        self.data = ABSA_DataModule(self, **kwargs)
        if 'tokeniser' in kwargs:
            self.tokenizer = kwargs['tokeniser']  # attribute expected by lightning
        else:
            # this happens when loading a checkpoint
            self.tokenizer = None  # TODO: this may break ability to use the model
        self.__build_model()
        self.__build_loss()
        # prepare training with frozen BERT layers so that the new
        # classifier head can first adjust to BERT before BERT
        # adjusts to the classifier in later epochs        
        if hparams.nr_frozen_epochs > 0:
            self.freeze_encoder()
        else:
            self._frozen = False
        self.nr_frozen_epochs = hparams.nr_frozen_epochs
        self.record_predictions = False
            
    def __build_model(self) -> None:
        ''' Init BERT model, tokeniser and classification head '''
        # Q: Why not use AutoModelForSequenceClassification?
        self.bert = AutoModel.from_pretrained(
            model_name,  # was: self.hparams.encoder_model
            output_hidden_states = True
        )
        # parameters for the classification head: best values
        # depend on the task and dataset; the below values
        # have not been tuned much but work reasonable well
        self.classification_head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.bert.config.hidden_size, 1536),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(1536, 256),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(256, self.data.label_encoder.vocab_size)
        )
        
    def __build_loss(self):
        self._loss = nn.CrossEntropyLoss()

In [19]:
import logging as log

class Classifier_part_2(Classifier_part_1):
    
    def unfreeze_encoder(self) -> None:
        if self._frozen:
            log.info('\n== Encoder model fine-tuning ==')
            for param in self.bert.parameters():
                param.requires_grad = True
            self._frozen = False
            
    def freeze_encoder(self) -> None:
        for param in self.bert.parameters():
            param.requires_grad = False
        self._frozen = True

    def predict(self, sample: dict) -> dict:
        ''' make a prediction for a single data instance '''
        if self.training:
            self.eval()
        with torch.no_grad():
            batch_inputs, _ = self.prepare_sample(
                [sample],
                prepare_target = False
            )
            model_out = self.forward(batch_inputs)
            logits = torch.Tensor.cpu(model_out["logits"]).numpy()
            predicted_labels = [
                self.data.label_encoder.index_to_token[prediction]
                for prediction in numpy.argmax(logits, axis=1)
            ]
            sample["predicted_label"] = predicted_labels[0]
        return sample
    
    # functionality to obtain predictions for a dataset as a
    # side effect of asking PyTorch Lightning to get evaluation
    # results for a dataset
    # (the framework does not seem to provide a function to get
    # all predictions for a dataset)
    
    def start_recording_predictions(self):
        self.record_predictions = True
        self.reset_recorded_predictions()
        
    def stop_recording_predictions(self):
        self.record_predictions = False
        
    def reset_recorded_predictions(self):
        self.seq2label = {}

In [20]:
from torchnlp.utils import lengths_to_mask

class Classifier_part_3(Classifier_part_2):
    
    def forward(self, batch_input):
        tokens  = batch_input['input_ids']
        lengths = batch_input['length']
        mask = batch_input['attention_mask']
        # Run BERT model.
        word_embeddings = self.bert(tokens, mask).last_hidden_state
        sentemb = word_embeddings[:,0]  # at position of [CLS]
        logits = self.classification_head(sentemb)
        # Hack to conveniently use the model and trainer to
        # get predictions for a test set:
        if self.record_predictions:
            logits_np = torch.Tensor.cpu(logits).numpy()
            predicted_labels = [
                self.data.label_encoder.index_to_token[prediction]
                for prediction in numpy.argmax(logits_np, axis=1)
            ]
            for index, input_token_ids in enumerate(tokens):
                key = torch.Tensor.cpu(input_token_ids).numpy().tolist()
                # truncate trailing zeros
                while key and key[-1] == 0:
                    del key[-1]
                self.seq2label[tuple(key)] = predicted_labels[index]
        return {"logits": logits}
    
    def loss(self, predictions: dict, targets: dict) -> torch.tensor:
        """
        Computes Loss value according to a loss function.
        :param predictions: model specific output. Must contain a key 'logits' with
            a tensor [batch_size x 1] with model predictions
        :param labels: Label values [batch_size]
        Returns:
            torch.tensor with loss value.
        """
        return self._loss(predictions["logits"], targets["labels"])
    
    def add_missing_hparams(self):
        # fix various attribute errors when instantiating via
        # load_from_checkpoint()
        # like self.hparams.update() but do not overwrite 
        # values already set
        # TODO: check docs whether there is a parameter to
        #       request this behaviour from update()
        global hparams
        for key in hparams:
            if not hasattr(self.hparams, key):
                self.hparams[key] = hparams[key]

In [21]:


class Classifier_part_4(Classifier_part_3):
    
    def prepare_sample(self, sample: list, prepare_target: bool = True) -> (dict, dict):
        """ prepare a batch of instances to pass them into the model
        
        :param sample: list of dictionaries.
        
        Returns:
            - dictionary with the expected model inputs.
            - dictionary with the expected target labels.
        """
        assert len(sample) <= batch_size
        assert self.tokenizer is not None
        batch_seq_A = []
        batch_seq_B = []
        for item in sample:
            batch_seq_A.append(item['seq_A'])
            batch_seq_B.append(item['seq_B'])
        # run the tokeniser
        encoded_batch = self.tokenizer(
            batch_seq_A,
            batch_seq_B,
            is_split_into_words = True,
            return_length       = True,
            padding             = 'max_length',
            # https://github.com/huggingface/transformers/issues/8691
            return_tensors      = 'pt',
        )
        if not prepare_target:
            return encoded_batch, {}  # no target labels requested
        # Prepare target:
        batch_labels = []
        for item in sample:
            batch_labels.append(item['label'])
        assert len(batch_labels) <= batch_size
        try:
            targets = {
                "labels": self.data.label_encoder.batch_encode(batch_labels)
            }
            return encoded_batch, targets
        except RuntimeError:
            raise Exception("Label encoder found an unknown label.")


In [22]:


from collections import OrderedDict

class Classifier_part_5(Classifier_part_4):
    
    def training_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        ''' perform a training step with the given batch '''
        inputs, targets = batch
        model_out = self.forward(inputs)
        loss_val = self.loss(model_out, targets)
        # in DP mode (default) make sure if result is scalar, there's another dim in the beginning
        # Q: What is this about?
        # attribute no longer exists
        #if self.trainer.use_dp or self.trainer.use_ddp2:
        #    loss_val = loss_val.unsqueeze(0)
        output = OrderedDict({"loss": loss_val})
        self.log('train_loss', loss_val, on_step=True, on_epoch=True, prog_bar=True)
        # can also return just a scalar instead of a dict (return loss_val)
        return output
   
    def test_or_validation_step(self, test_type, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        ''' perform a test or validation step with the given batch '''
        inputs, targets = batch
        model_out = self.forward(inputs)
        loss_val = self.loss(model_out, targets)
        y = targets["labels"]
        # get predictions
        y_hat = model_out["logits"]
        labels_hat = torch.argmax(y_hat, dim=1)
        # get accuracy
        val_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)
        val_acc = torch.tensor(val_acc)
        if self.on_gpu:
            val_acc = val_acc.cuda(loss_val.device.index)
        # in DP mode (default) make sure if result is scalar, there's another dim in the beginning
        # attribute no longer exists
        #if self.trainer.use_dp or self.trainer.use_ddp2:
        #    loss_val = loss_val.unsqueeze(0)
        #    val_acc = val_acc.unsqueeze(0)
        output = OrderedDict({
            test_type + "_loss": loss_val,
            test_type + "_acc":  val_acc,
            'batch_size': len(batch),
        })
        return output
    
    def validation_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        return self.test_or_validation_step(
            'val', batch, batch_nb, *args, **kwargs
        )
    
    def test_step(self, batch: tuple, batch_nb: int, *args, **kwargs) -> dict:
        return self.test_or_validation_step(
            'test', batch, batch_nb, *args, **kwargs
        )



In [23]:
from torch import optim

class Classifier(Classifier_part_5):
    
    # validation_end() is now validation_epoch_end()
    # https://github.com/PyTorchLightning/pytorch-lightning/blob/efd272a3cac2c412dd4a7aa138feafb2c114326f/CHANGELOG.md
    
    def test_or_validation_epoch_end(self, test_type, outputs: list) -> None:
        ''' calculate average loss and accuracy over all batches,
            reducing the weight of the last batch according to its
            size so that all data instances have equal influence
            on the scores
        '''
        val_loss_mean = 0.0
        val_acc_mean = 0.0
        total_size = 0
        for output in outputs:
            val_loss = output[test_type + "_loss"]
            # reduce manually when using dp
            # -- attribute no longer exists
            #if self.trainer.use_dp or self.trainer.use_ddp2:
            #    val_loss = torch.mean(val_loss)
            val_loss_mean += val_loss
            # reduce manually when using dp
            val_acc = output[test_type + "_acc"]
            #if self.trainer.use_dp or self.trainer.use_ddp2:
            #    val_acc = torch.mean(val_acc)
            # We weight the batch accuracy by batch size to not give
            # higher weight to the items of a smaller, final bacth.
            batch_size = output['batch_size']
            val_acc_mean += val_acc * batch_size
            total_size += batch_size
        val_loss_mean /= len(outputs)
        val_acc_mean /= total_size
        self.log(test_type+'_loss', val_loss_mean)
        self.log(test_type+'_acc',  val_acc_mean)

    def validation_epoch_end(self, outputs: list) -> None:
        self.test_or_validation_epoch_end('val', outputs)
                                     
    def test_epoch_end(self, outputs: list) -> None:
        self.test_or_validation_epoch_end('test', outputs)
        
    def configure_optimizers(self):
        """ Sets different Learning rates for different parameter groups. """
        parameters = [
            {"params": self.classification_head.parameters()},
            {
                "params": self.bert.parameters(),
                "lr": self.hparams.encoder_learning_rate,
                #"weight_decay": 0.01,  # TODO: try this as it is in the BERT paper
            },
        ]
        optimizer = optim.Adam(parameters, lr=self.hparams.learning_rate)
        return [optimizer], []

    def on_epoch_end(self):
        """ Pytorch lightning hook """
        if self.current_epoch + 1 >= self.nr_frozen_epochs:
            self.unfreeze_encoder()

## 4.1 Training

In [24]:
classifier = Classifier(
    hparams = hparams,    
    # parameters for ABSA_DataModule:
    data_split = (tr_dataset_object, dev_dataset_combined, te_dataset_combined),
    # additional required parameters:
    tokeniser  = tokeniser
)   
print('Ready.')

Labelset: {'neutral', 'positive', 'negative'}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Ready.


In [25]:
# https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import os
import time

early_stop_callback = EarlyStopping(
    monitor   = 'val_acc',
    min_delta = 0.00,
    patience  = 7,
    verbose   = False,
    mode      = 'max',
)

save_top_model_callback = ModelCheckpoint(
    save_top_k = 3,
    monitor    = 'val_acc',
    mode       = 'max',
    filename   = '{val_acc:.4f}-{epoch:02d}-{val_loss:.4f}'
)

trainer = pl.Trainer(
    callbacks=[early_stop_callback, save_top_model_callback],
    max_epochs = max_epochs,
    min_epochs = classifier.hparams.nr_frozen_epochs + 2,
    gpus = classifier.hparams.gpus,
    accumulate_grad_batches = accumulate_grad_batches,
    limit_train_batches     = limit_train_batches,
    check_val_every_n_epoch = 1,
    # https://github.com/PyTorchLightning/pytorch-lightning/issues/6690
    logger = pl.loggers.TensorBoardLogger(os.path.abspath('lightning_logs')),
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [26]:
start = time.time()
trainer.fit(classifier, classifier.data)
print('Training time: %.0f minutes' %((time.time()-start)/60.0))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type             | Params
---------------------------------------------------------
0 | bert                | BertModel        | 109 M 
1 | classification_head | Sequential       | 1.6 M 
2 | _loss               | CrossEntropyLoss | 0     
---------------------------------------------------------
1.6 M     Trainable params
109 M     Non-trainable params
111 M     Total params
444.231   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Training time: 43 minutes


In [27]:
# https://www.youtube.com/watch?v=nCq_vy9qE-k at 44:59

%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 18785), started 5 days, 23:55:18 ago. (Use '!kill 18785' to kill it.)

In [28]:
print('The best model is', save_top_model_callback.best_model_path)

print('Best validation set accuracy:', save_top_model_callback.best_model_score)

# The following automatically loads the best weights according to
# https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html

print('Test results via trainer.test():')
results = trainer.test()  # also prints results as a side effect



The best model is /home/jwagner/sentiment-2021/absa-rationale-eval/notebooks/lightning_logs/default/version_80/checkpoints/val_acc=0.8821-epoch=06-val_loss=0.3999.ckpt
Best validation set accuracy: tensor(0.8821, device='cuda:0')
Test results via trainer.test():


/home/jwagner/sentiment-2021/absa-rationale-eval/venv-pytorch/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7853130102157593, 'test_loss': 0.5674376487731934}
--------------------------------------------------------------------------------


## 5.1 Save Best Model outside Logs

Rather than manually locating the best model in the lightning logs folder and copying it to another location, use the library to save a copy. This also gives us the option to save a copy without the training state of the Adam optimiser, reducing model size by about 67%, training parameters and filesystem paths that we may not want to share with users of the model.


In [29]:
# https://pytorch-lightning.readthedocs.io/en/latest/common/weights_loading.html

# after just having run test(), the best checkpoint is still loaded but that's
# not a documented feature so to be on the safe side for future versions we
# need to explicitly load the best checkpoint:

best_model = Classifier.load_from_checkpoint(
    checkpoint_path = trainer.checkpoint_callback.best_model_path
    # the hparams including hparams.batch_size appear to have been
    # saved in the checkpoint automatically
)
# best_model.save_checkpoint('best.ckpt') does not exist
# --> need to wrap model into trainer to be able to save a checkpoint

new_trainer = pl.Trainer(
    resume_from_checkpoint = trainer.checkpoint_callback.best_model_path,
    gpus = -1,  # avoid warnings (-1 = automatic selection)
    # https://github.com/PyTorchLightning/pytorch-lightning/issues/6690
    logger = pl.loggers.TensorBoardLogger(os.path.abspath('lightning_logs')),
)
new_trainer.model = best_model  # @model.setter in plugins/training_type/training_type_plugin.py

new_trainer.save_checkpoint(
    "best-model-weights-only.ckpt",
    True,  # save_weights_only
    # (if saved with setting the 2nd arg to True, the checkpoint   # TODO: "False"?
    # will contain absoulte paths and training parameters)
)

# to just save the bert model in pytorch format and without the classification head, we follow
# https://github.com/PyTorchLightning/pytorch-lightning/issues/3096#issuecomment-686877242
best_model.bert.save_pretrained('best-bert-encoder.pt')

# TODO: the above only saves the BERT encoder, not the classification head

# Since the lightning module inherits from pytorch, we can save the full network in
# pytorch format:
torch.save(best_model.state_dict(), 'best-model.pt')

print('Ready')

Labelset: {'neutral', 'positive', 'negative'}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Ready


## 5.2 Load and Test Model

In [30]:
best_model = Classifier.load_from_checkpoint(
    checkpoint_path = 'best-model-weights-only.ckpt'
)

best_model.eval()  # enter prediction mode, e.g. turn off dropout

print(best_model.data.data_split)  # confirm the data is not saved

if best_model.tokenizer is None:
    #print('setting tokeniser')
    best_model.tokenizer = tokeniser

def test_and_print(te_dataset_object):
    test_dataloader = DataLoader(
        dataset     = te_dataset_object,
        batch_size  = best_model.hparams.batch_size,
        collate_fn  = best_model.prepare_sample,
        num_workers = best_model.hparams.loader_workers,
    )
    #print('number of batches:', len(test_dataloader))
    new_trainer = pl.Trainer(
        gpus = -1,
        # https://github.com/PyTorchLightning/pytorch-lightning/issues/6690
        logger = pl.loggers.TensorBoardLogger(os.path.abspath('lightning_logs')),
    )
    result = new_trainer.test(best_model, test_dataloaders = [test_dataloader])
    assert len(result) == 1
    return result[0]['test_acc']
    
def get_subset_by_domain(dataset_object, domain):
    indices = []
    for index in range(len(dataset_object)):
        if domain == dataset_object[index]['domain']:
            indices.append(index)
    return torch.utils.data.Subset(dataset_object, indices)

Labelset: {'neutral', 'positive', 'negative'}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(None, None, None)


In [31]:
summary = []
header = []
header.append('SeqB')
header.append('Q')
header.append('     Overall')
if len(domains) > 1:
    for domain in sorted(list(domains)):
        header.append('%12s' %(domain.title()))
header.append('Description')
summary.append('\t'.join(header))

mask2seqb = {
    None: 'Full',
    'O':  'SE',
    'I':  'Other',
}

last_seqb = None
for te_index, te_dataset_object in enumerate(te_dataset_objects):
    template_index = te_index % len(templates)
    row = []
    seqb = mask2seqb[te_dataset_object.mask]
    row.append(seqb)
    if seqb != last_seqb:
        print('Testing', seqb)
        last_seqb = seqb
    row.append('%d' %template_index)
    question = templates[template_index]['question']
    print('\nQuestion template %d: %r' %(template_index, question))
    score = 100.0 * test_and_print(te_dataset_object)
    row.append('%.9f' %score)
    if len(domains) > 1:
        print('\nBreakdown by domain:')
        for domain in sorted(list(domains)):
            print(domain)
            score = 100.0 * test_and_print(
                get_subset_by_domain(te_dataset_object, domain)
            )
            row.append('%.9f' %score)
    print()
    row.append(templates[template_index]['description'])
    summary.append('\t'.join(row))
print('\nSummary:')
print('\n'.join(summary))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/jwagner/sentiment-2021/absa-rationale-eval/venv-pytorch/lib/python3.6/site-packages/pytorch_lightning/trainer/trainer.py:680: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing Full

Question template 0: '%(domain)s: What do you think of the %(attribute_label)s of %(entity_type)s?'


Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.853365421295166, 'test_loss': 0.4615432918071747}
--------------------------------------------------------------------------------

Breakdown by domain:
laptop


...

Testing: 0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6547029614448547, 'test_loss': 0.7728490233421326}
--------------------------------------------------------------------------------
restaurant


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7384259104728699, 'test_loss': 0.6286464929580688}
--------------------------------------------------------------------------------


Summary:
SeqB	Q	     Overall	      Laptop	  Restaurant	Description
Full	0	85.336542130	83.910888433	86.728394032	Sun et al. QA-M
SE	0	80.588942766	76.113861799	84.876543283	Sun et al. QA-M
Other	0	69.651442766	65.470296144	73.842591047	Sun et al. QA-M


# 6 Saliency Map
## 6.1 Copy of Test Data with Prediction as Label

The EMNLP 2020 interpretability tutorial suggests to implement a function `pred2label()` that creates a new instance `(x,y)` with the label `y` set to the model's prediction for `x`.

In [32]:
# for tackling the more diffcult step first, we can just use the gold labels

pred_dataset_objects = te_dataset_objects

## 6.2 Get Input Gradient
For each data instance, get the average gradient over a few samples near the ...

In [45]:
def get_embedding_layer(model):
    return model.bert.embeddings
    #return model.bert.embeddings.word_embeddings

In [46]:
# EMNLP 2020 interpretatibility video at 3:09:52
# with modifications to 

def get_gradients(model, instances, labels):
    embedding_gradients = []
    def grad_hook(module, grad_in, grad_out):
        embedding_gradients.append(grad_out[0])
    embedding = get_embedding_layer(model)
    handle = embedding.register_full_backward_hook(grad_hook)
    try:
        predictions = model(instances)
        # TODO: should we use "logits" directly to
        #       "avoid saturation issues"?
        loss = model.loss(predictions, labels)
        loss.backward()
    except:
        handle.remove()
        raise Exception
    handle.remove()
    return embedding_gradients

best_model.unfreeze_encoder()   # otherwise gradients will not arrive at embedding layer

finalised_instance, labels = best_model.prepare_sample(
    sample = [
        pred_dataset_objects[0][0],  # first set = "Full", first item
    ],
    prepare_target = True
)

#print(finalised_instance)
#print(labels)  # numeric (encoded) labels

gradients = get_gradients(best_model, finalised_instance, labels)
print(gradients)
# 

[tensor([[[-1.1985e-05, -1.0283e-05,  3.1617e-05,  ...,  1.8402e-05,
           1.1905e-05,  2.3714e-05],
         [-9.6155e-06,  2.2740e-06,  1.3776e-06,  ..., -1.5271e-05,
          -6.5746e-06, -8.9930e-06],
         [-1.3327e-06,  3.7620e-06,  1.7714e-06,  ..., -4.5609e-06,
          -1.2407e-06, -4.5818e-06],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]])]


In [47]:
print(len(gradients))
print(gradients[0].shape)

1
torch.Size([1, 512, 768])


In [73]:
# next step is then at 3:12:38 to add a forward hook and calculate the saliencies

def dot_and_normalise(gradients, forward_embeddings):
    retval = []
    for gradient, forward_embedding in zip(
        gradients, forward_embeddings
    ):
        # https://discuss.pytorch.org/t/how-to-do-elementwise-multiplication-of-two-vectors/13182
        products = gradient * forward_embedding
        # adjusted from
        # https://github.com/PAIR-code/lit/blob/main/lit_nlp/components/gradient_maps.py
        # GradientNorm._interpret()
        norms = torch.linalg.norm(products, axis=2)
        normalised_norms = norms / torch.sum(norms)
        retval.append(normalised_norms)
    return retval
    
def interpret(model, instances, labels):
    forward_embeddings = []
    def forward_hook(module, inputs, outputs):
        forward_embeddings.append(outputs)
    embedding = get_embedding_layer(model)
    handle = embedding.register_forward_hook(forward_hook)
    gradients = get_gradients(model, instances, labels)
    handle.remove()
    saliencies = dot_and_normalise(
        gradients,
        forward_embeddings,
    )
    return saliencies, gradients, forward_embeddings

In [81]:
import numpy

s,g,f = interpret(best_model, finalised_instance, labels)
print(len(g), len(f), len(s))
print(g[0].shape, f[0].shape, s[0].shape)
dp = g[0] * f[0]
print(dp.shape)
for t in (g[0], f[0]):
    norms = torch.linalg.norm(t, axis=2)
    print(norms.shape)
    
for i in range(40):
    print(s[0][0][i], finalised_instance[0].tokens[i])

1 1 1
torch.Size([1, 512, 768]) torch.Size([1, 512, 768]) torch.Size([1, 512])
torch.Size([1, 512, 768])
torch.Size([1, 512])
torch.Size([1, 512])
tensor(0.0281, grad_fn=<SelectBackward>) [CLS]
tensor(0.0446, grad_fn=<SelectBackward>) laptop
tensor(0.0193, grad_fn=<SelectBackward>) :
tensor(0.0138, grad_fn=<SelectBackward>) what
tensor(0.0096, grad_fn=<SelectBackward>) do
tensor(0.0116, grad_fn=<SelectBackward>) you
tensor(0.0165, grad_fn=<SelectBackward>) think
tensor(0.0102, grad_fn=<SelectBackward>) of
tensor(0.0135, grad_fn=<SelectBackward>) the
tensor(0.0415, grad_fn=<SelectBackward>) general
tensor(0.0194, grad_fn=<SelectBackward>) of
tensor(0.0482, grad_fn=<SelectBackward>) laptop
tensor(0.0168, grad_fn=<SelectBackward>) ?
tensor(0.0116, grad_fn=<SelectBackward>) [SEP]
tensor(0.0169, grad_fn=<SelectBackward>) like
tensor(0.0127, grad_fn=<SelectBackward>) so
tensor(0.0179, grad_fn=<SelectBackward>) many
tensor(0.0225, grad_fn=<SelectBackward>) others
tensor(0.0146, grad_fn=<Selec

## Appendix A: Example BERT Tokenisation

In [36]:
example_batch = []
for i in (0, 4, 8, 18):  # select a few interesting instances
    example_batch.append(tr_dataset[i][3])   

tokenised_text = tokeniser(
    example_batch,
    is_split_into_words = True,
)

for i, token_ids in enumerate(tokenised_text['input_ids']):
    if i: print()
    print(i, '\tinput:        ', example_batch[i])
    print(   "\t['input_ids']:", token_ids)
    print(   '\ttokens:       ', tokeniser.convert_ids_to_tokens(token_ids))

0 	input:         ['All', 'I', 'can', 'say', 'is', 'W-O-W', '.']
	['input_ids']: [101, 2035, 1045, 2064, 2360, 2003, 1059, 1011, 1051, 1011, 1059, 1012, 102]
	tokens:        ['[CLS]', 'all', 'i', 'can', 'say', 'is', 'w', '-', 'o', '-', 'w', '.', '[SEP]']

1 	input:         ['I', 'bought', 'a', 'protector', 'for', 'my', 'key', 'pad', 'and', 'it', 'works', 'great', ':)']
	['input_ids']: [101, 1045, 4149, 1037, 16167, 2005, 2026, 3145, 11687, 1998, 2009, 2573, 2307, 1024, 1007, 102]
	tokens:        ['[CLS]', 'i', 'bought', 'a', 'protector', 'for', 'my', 'key', 'pad', 'and', 'it', 'works', 'great', ':', ')', '[SEP]']

2 	input:         ['The', 'mouse', 'is', 'a', 'little', 'bit', 'different', 'than', 'what', 'you', "'re", 'used', 'to', 'though-', 'it', 'has', 'one', 'big', 'flat', 'panel', 'and', 'one', 'full', 'bar', '(', 'instead', 'of', 'two', 'separate', 'ones', ')', 'to', 'click', 'with-', 'but', 'you', 'get', 'used', 'to', 'it', 'quite', 'quickly', '.']
	['input_ids']: [101, 1996, 80

## Appendix B: Sequence Length Distribution

In [37]:
from collections import defaultdict
    
bin_width = 10

for is_not_first, p_domain in enumerate(domains):
    if is_not_first:
        print()
    print(p_domain)
    distribution = defaultdict(lambda: 0)
    tr_dataset.append(10*[None])  # hack to simplify loop below
    batch = []
    labels = []
    max_length_bin = 0
    for domain, _, text, _, _, _, _, _, _, label in tr_dataset:
        if domain != p_domain:
            continue
        if text is not None:
            batch.append(text)
            labels.append(label)
        if len(batch) == batch_size \
        or (text is None and len(batch) > 0):
            tokenised_batch = tokeniser(
                batch,
                is_split_into_words = False,
            )
            for index, token_ids in enumerate(tokenised_batch['input_ids']):
                label = labels[index]
                length = len(token_ids)
                length_bin = length // bin_width
                distribution[(label,   length_bin)] += 1
                distribution[('total', length_bin)] += 1
                if length_bin > max_length_bin:
                    max_length_bin = length_bin
            batch = []
            labels = []
    del tr_dataset[-1]  # remove "end of data" marker of hack above   
    header = []
    header.append('LengthBin')
    for polarity in sorted(tr_observed_polarities):
        header.append('%12s' %polarity)
    header.append('%12s' %'Total')
    header.append('%12s' %'Positivity')
    print('\t'.join(header))
    for length_bin in range(0, max_length_bin+1):
        row = []
        row.append('%4d-%4d' %(
            bin_width*length_bin,
            bin_width*(1+length_bin)-1
        ))
        total = 0
        for label in sorted(tr_observed_polarities):
            count = distribution[(label, length_bin)]
            row.append('%12d' %count)
            total += count
        row.append('%12d' %total)
        if total:
            row.append('%10.0f%%' %(100.0*distribution[('positive', length_bin)]/float(total)))
        else:
            row.append('%11s' %'n/a')
        print('\t'.join(row))

laptop
LengthBin	    negative	     neutral	    positive	       Total	  Positivity
   0-   9	         116	          18	         269	         403	        67%
  10-  19	         464	          72	         708	        1244	        57%
  20-  29	         284	          54	         399	         737	        54%
  30-  39	          99	          17	         118	         234	        50%
  40-  49	          43	          14	          38	          95	        40%
  50-  59	          11	           2	          19	          32	        59%
  60-  69	           6	           0	           0	           6	         0%
  70-  79	           3	           0	           3	           6	        50%
  80-  89	           3	           0	           0	           3	         0%

restaurant
LengthBin	    negative	     neutral	    positive	       Total	  Positivity
   0-   9	          73	          12	         259	         344	        75%
  10-  19	         288	          46	         692	        1026	        67%
  20-  29	       

## Appendix C: Example Dataset Object

In [38]:
dataset_obj = ABSA_Dataset(
    tr_dataset,
    put_question_first = put_question_first,
    # template_index = -1,   # -1 = random pick
    template_index = 0,
    mask = 'O'
    
)
for i in range(2):
    print(i, dataset_obj[i])
    print(i, dataset_obj[i])  # repeat call doesn't give the same result with template_index = -1
    print(i, dataset_obj[i])

0 {'seq_A': ['laptop:', 'What', 'do', 'you', 'think', 'of', 'the', 'GENERAL', 'of', 'LAPTOP?'], 'seq_B': ['[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'W-O-W', '[MASK]'], 'label': 'positive', 'domain': 'laptop', 'mask': 'O', 'info': None}
0 {'seq_A': ['laptop:', 'What', 'do', 'you', 'think', 'of', 'the', 'GENERAL', 'of', 'LAPTOP?'], 'seq_B': ['[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'W-O-W', '[MASK]'], 'label': 'positive', 'domain': 'laptop', 'mask': 'O', 'info': None}
0 {'seq_A': ['laptop:', 'What', 'do', 'you', 'think', 'of', 'the', 'GENERAL', 'of', 'LAPTOP?'], 'seq_B': ['[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'W-O-W', '[MASK]'], 'label': 'positive', 'domain': 'laptop', 'mask': 'O', 'info': None}
1 {'seq_A': ['laptop:', 'What', 'do', 'you', 'think', 'of', 'the', 'GENERAL', 'of', 'LAPTOP?'], 'seq_B': ['[MASK]', '[MASK]', '[MASK]', 'insane', '[MASK]'], 'label': 'positive', 'domain': 'laptop', 'mask': 'O', 'info': None}
1 {'seq_A': ['laptop:', 'What', 'do', 'you'

## Appendix D: Location of Instances Picked for Devset
Showing first 750 instances only.

In [39]:
row = []
n = len(tr_indices) + len(dev_indices)
for i in range(n):
    if i in dev_indices:
        row.append('d')
    else:
        row.append('_')
    if len(row) == 75 or i+1 == n:
        print(''.join(row))
        row = []
    if i > 750:
        break

___________d_____________d_________dd____d______________d__________________
___________d_______________d________________________d______________________
__d________d_______________d______d________________________________d______d
__________________________________________d___________________________d____
d____________________________________d___________d_______d_________________
_____________________________________d____d____________________________d___
__________d______d_________________________________dd______d_______________
_______________________________________________d____d____________________d_
__________________________d_______d______________d_________________________
______________________d______________________________________d__________d__
